## Modeling the elastic, plastic and viscous deformation

We will use a [ASPECT](https://aspect.geodynamics.org/) to demonstrate the differences in the elastic, plastic and viscous deformation under a similar model setups. For each of the deformation styles, a different parameter file describes the model deformation parameters. We will first go over some of the key parameters used to describe the model setup and then go over the key differences between the material properties employed in the three models.

#### Geometry Model

The model is a 2D box with dimensions 50x200 km in width and height, respectively. The repetitions along X and Y ensure cell size that has an aspect ratio close to 1.

```
subsection Geometry model
  set Model name = box

  subsection Box
    set X repetitions = 2
    set Y repetitions = 8
    set X extent      = 50e3
    set Y extent      = 200e3
  end
end
```

#### Initial Compositional Field

We define a compositional field, `slab`, that deforms differently based on its material properties. The slab represents a rectangular region along the right side of the model domain such that the slab is 100 km in height and 35 km in width. The slab is defined using a hyperbolic tangent function to ensure a smooth transition between the slab and the surrounding mantle material.

```
subsection Compositional fields
  set Number of fields = 1
  set Names of fields  = slab
end

subsection Initial composition model
  set Model name = function
  subsection Function
    set Variable names      = x,y
    set Function constants  = x0=15e3,  
    set Function expression = if (y>50e3 && y<150e3,  0.5+0.5*tanh((x-x0)/1000), 0)
  end
end
```

> Note: For plastic and elastic deformations, we use additional compositional fields to track the stress tensor components, but they are not used in computing the material properties of the slab.

#### Gravity Model
We apply gravity for the first 8,000 years and then remove it to investigate the deformation of slab with time and any subsequent relaxation of the slab.

```
subsection Gravity model
  set Model name = function
  subsection Function
    set Variable names = x,y,t
    set Coordinate system = depth
    set Function expression = 0; if (t < 8e3, -10, 0)
  end
end
```

#### Velocity Boundary Conditions

We fix the right edge of the box and keep the other boundaries free-slip. This allows us to investigate the deformation within the slab due to gravity.

```
subsection Boundary velocity model
  set Tangential velocity boundary indicators = bottom, top, left
  set Zero velocity boundary indicators = right
end

```

#### Initial Temperature Model
We set a constant temperature of 20 degrees Celsius throughout the model domain, which means that we are only simulating the advection of compositional field of interest, i.e., slab.

```
subsection Initial temperature model
  set Model name = function
  subsection Function
    set Function expression = 293
  end
end
```

### Elastic Deformation—elasticity.prm

#### Formulation
To include the right hand side in the momentum equation, we need to enable elasticity.

```
subsection Formulation
  set Enable elasticity = true
end
```

#### Material Properties
We use a higher density and lower shear modulus for the slab compared to the surrounding medium to ensure that it bends under gravity. We also ensure that the surrounding medium has a lower viscosity than the slab so that the slab does not 'flow' but rather deforms elastically.

```
# Material model
subsection Material model
  set Model name = viscoelastic

  subsection Viscoelastic
    set Densities            = 3200, 3200, 3200, 3200, 3200, 3200, 3200, 3400
    set Viscosities          = 1.e19, 1.e19, 1.e19, 1.e19, 1.e19, 1.e19, 1.e19, 1.e21
    set Elastic shear moduli = 1.e14, 1.e14, 1.e14, 1.e14, 1.e14, 1.e14, 1.e14, 80.e7
    set Fixed elastic time step     = 1e4
    set Use fixed elastic time step = true
    set Viscosity averaging scheme  =  arithmetic
    set Thermal expansivities = 0
  end
end
```
Here, the first 7 terms in the `Densities`, `Viscosities` and `Elastic shear moduli` correspond to the background medium and the 6 compositional fields used to track the stress tensor components, respectively. The last term corresponds to the slab.  

### Viscous Deformation—viscous.prm

#### Material Properties
We use a denser and weaker slab compared to the surrounding mantle to ensure that it 'flows' under gravity.

```
subsection Material model
  set Model name = simple

  subsection Simple model
    set Density differential for compositional field 1 = 200
    set Viscosity            = 1e19
    set Composition viscosity prefactor = 0.1
  end
end
```

### Plastic Deformation—plastic.prm


#### Material Properties

We use a denser slab with lower cohesion compared to the surrounding mantle to ensure that it yields under gravity.

```
subsection Material model

  set Model name = visco plastic

  subsection Visco Plastic
    set Viscous flow law =  dislocation

    set Densities                                 = 3200, 3400
    set Prefactors for dislocation creep          = 5e-22
    set Stress exponents for dislocation creep    = 1.0
    set Activation energies for dislocation creep = 0.0
    set Activation volumes for dislocation creep  = 0.0

    set Maximum viscosity = 1.e22
    set Minimum viscosity = 1.e17

    set Elastic shear moduli = 1e11
    set Fixed elastic time step     = 1e3
    set Use fixed elastic time step =  false    
   
    set Angles of internal friction = 30
    set Angles of dilation          = 30
    set Cohesions                   = 10e10, 10.e5
    set Allow negative pressures in plasticity = true
  end
end
```
Just like the elastic model, the first terms correspond to the background medium and the last term corresponds to the slab. The cohesion for the slab is set much lower than that of the surrounding medium to ensure yielding under gravity.

 &nbsp;<div style="text-align: right">  
    &rarr; <b>NEXT: [Plot simulated deformations](./6_plot_simulated_deformation_types.ipynb) </b> <a href=""></a> &nbsp;&nbsp;
     <img src="../assets/education-gem-notebooks_icon.png" alt="icon"  style="width:4%">
  </div>